In [ ]:

import numpy as np
import pybamm

import gurobipy as gp
import pandas as pd

from nemosis import dynamic_data_compiler
from blast import models, utils


In [1]:

import pandas as pd
import numpy as np

schedule = pd.read_csv("schedule_year1.csv")
schedule["Time_s"] = schedule.index * 5 * 60
schedule["temperature_C"] = 25
data = {
    "Time_s": schedule["Time_s"].to_numpy(),
    "SOC": schedule["SOC_%"].to_numpy(),
    "Temperature_C": np.repeat([25], len(schedule))
}

# results = simulate_battery(schedule)

In [29]:
from blast import models
dt = 5/60.0
data = {
    "Time_s": schedule["Time_s"].to_numpy(),
    "SOC": schedule["SOC_%"].to_numpy(),
    "Temperature_C": schedule["temperature_C"].to_numpy()
}
batt = models.Lfp_Gr_250AhPrismatic(degradation_scalar=1.1901)
batt.simulate_battery_life(data, threshold_capacity=0.7, is_conserve_energy_throughput=False)

dSOC = schedule['SOC'].diff().fillna(0)
efc = 0.5 * dSOC.abs().sum() * ((365*24*3600) / data['Time_s'][-1])

batt_soc = batt.stressors['soc'][1:]
soc = batt_soc.tolist()
batt_dod = batt.stressors['dod'][1:]
dod = batt_dod.tolist()
efc_subcycle = batt.stressors['delta_efc'][1:].tolist()
t_subcycle = batt.stressors['delta_t_days'][1:].tolist()
dod_rms =  np.sqrt(np.mean(batt_dod**2))
dod_median = np.median(batt_dod)
dod_95 = np.percentile(batt_dod, 95)
soc_mean = np.mean(batt_soc)
soc_median = np.median(batt_soc)

results = pd.DataFrame({
    "Relative Capacity": batt.outputs['q'],
    "EFC": batt.stressors["efc"],
    "T (Days)": batt.stressors['t_days'],
})


summary = {
    'EFCs/year': [efc],
    'Mean SOC': [soc_mean],
    'Median SOC': [soc_median],
    'DOD RMS': [dod_rms],
    'Median DOD': [dod_median],
    'DOD 95th percentile': [dod_95],
    "Lifetime": [batt.stressors['t_days'][-1]]
}


# Convert degradation time to seconds to match the dispatch dataframe
df_deg = results.copy()
df_disp = schedule.copy()
df_deg["Time_s"] = df_deg["T (Days)"] * 24 * 3600  # convert days → seconds

# 2. Determine total simulated lifetime in seconds (until 0.7 SoH)
lifetime_s = df_deg["Time_s"].iloc[-1]  # e.g., 3294 days → ~284 million seconds

# 3. Find how long one dispatch cycle lasts
period_s = df_disp["Time_s"].iloc[-1]  # duration of one dispatch cycle

# 4. Compute how many repeats needed to reach the lifetime duration
n_repeats = int(np.ceil(lifetime_s / period_s))

# 5. Repeat the dispatch schedule
df_disp = pd.concat(
    [df_disp.assign(Time_s=df_disp["Time_s"] + i * (period_s + df_disp["Time_s"].iloc[1]))
     for i in range(n_repeats)],
    ignore_index=True
)

# Interpolate the relative capacity to the dispatch time base
df_disp["Relative Capacity"] = np.interp(
    df_disp["Time_s"],
    df_deg["Time_s"],
    df_deg["Relative Capacity"]
)
df_disp["EFC"] = np.interp(
    df_disp["Time_s"],
    df_deg["Time_s"],
    df_deg["EFC"]
)

# Optional: compute degraded net power (useful for lifetime revenue estimation)
df_disp["degraded_P_net"] = df_disp["P_net"] * df_disp["Relative Capacity"]
df_disp["degraded_P_chg"] = -df_disp["P_net"].clip(upper=0)
df_disp["degraded_P_dis"] = df_disp["P_net"].clip(lower=0)

# Optional: compute revenue
df_disp["revenue"] = df_disp["degraded_P_net"] * dt / 1000 * df_disp["price"]
df_disp["Datetime"] = pd.to_datetime(pd.Timestamp("2025-01-01 00:00:00") + pd.to_timedelta(df_disp["Time_s"], unit="s"))
df_disp = df_disp.drop(columns=["P_charge", "P_discharge", "P_net"])

daily_results = df_disp.groupby(df_disp["Datetime"].dt.date).aggregate({
    "revenue": "sum",
    "EFC": "last",
    "degraded_P_dis": "sum",
    "degraded_P_chg": "sum",
    "Relative Capacity": "last"
})

yearly_results = df_disp.groupby(df_disp["Datetime"].dt.year).aggregate({
    "revenue": "sum",
    "EFC": "last",
    "degraded_P_dis": "sum",
    "degraded_P_chg": "sum",
    "Relative Capacity": "last"
})

summary["Lifetime Value"] = df_disp["revenue"].sum()


In [31]:
daily_results

,revenue,EFC,degraded_P_dis,degraded_P_chg,Relative Capacity
Datetime,,,,,
2025-01-01,22.391957,3.227208,8865.805647,9041.108017,0.998931
2025-01-02,21.050212,6.107690,7761.872683,8386.165866,0.998359
2025-01-03,45.626252,8.109716,5429.070697,4930.521089,0.997998
2025-01-04,36.655069,10.882895,7429.232402,8116.511734,0.997599
2025-01-05,32.543697,14.044320,7902.056728,8211.296317,0.997150
...,...,...,...,...,...
2034-01-04,39.487613,7593.695642,5520.967034,6218.027980,0.700225
2034-01-05,41.521451,7596.138602,4841.619936,5123.464673,0.700152
2034-01-06,24.678543,7598.373895,8838.173343,8862.624433,0.700085


In [30]:
yearly_results

,revenue,EFC,degraded_P_dis,degraded_P_chg,Relative Capacity
Datetime,,,,,
2025,36499.835032,987.840511,2.700045e+06,2.880069e+06,0.936484
2026,34675.334908,1924.665788,2.699620e+06,2.876516e+06,0.894311
2027,33196.183715,2822.533738,2.702864e+06,2.880831e+06,0.858383
2028,31938.234564,3688.381218,2.708140e+06,2.886680e+06,0.826324
2029,30726.211111,4520.533211,2.702602e+06,2.881258e+06,0.797276
2030,29674.387055,5323.915260,2.701583e+06,2.881539e+06,0.770531
2031,28721.188889,6101.923023,2.702886e+06,2.881347e+06,0.745636
2032,27833.357959,6856.967540,2.708298e+06,2.886996e+06,0.722300
2033,26955.667086,7586.179874,2.703145e+06,2.881344e+06,0.700443


In [ ]:
from pathlib import Path
from stages.collate_optimisations import extract_metadata
import os
import argparse
from sched import scheduler
import pandas as pd
from pathlib import Path
import re

In [ ]:

def extract_metadata(path: Path):
    """Extract location, min_soc, max_soc, and cooling status from folder name."""
    pattern = r"(?P<location>[A-Za-z_]+)_min_soc=(?P<min>[\d\.]+)_max_soc=(?P<max>[\d\.]+).csv"
    match = re.search(pattern, path.name)
    if not match:
        return None
    return {
        "location": match.group("location"),
        "min_soc": float(match.group("min")),
        "max_soc": float(match.group("max")),
    }

In [ ]:
for file in Path("results/schedule").glob("Brisbane_*.csv"):
    print(extract_metadata(file))

In [ ]:
pd.read_csv("/workspace/results/simulation/Brisbane_min_soc=0_max_soc=0.6_cooling_disabled=true/daily_results.csv")

In [ ]:
schedule_list = []

for file in Path("results/schedule").glob("Brisbane_*.csv"):
    meta = extract_metadata(file)
    if meta is None:
        continue

    schedule_df = pd.read_csv(file)
    for k, v in meta.items():
        schedule_df[k] = v
    schedule_df["Name"] = schedule_df.apply(lambda row: f"{row["min_soc"]}-{row["max_soc"]}", axis=1)
    schedule_df["Datetime"] = pd.Timestamp("2024-01-01 00:00:00") + pd.to_timedelta(schedule_df["Time_s"], unit="s")

    # results_list.append(results_df)
    schedule_list.append(schedule_df.drop(columns=["Temperature_C", "location"]))

df = pd.concat(schedule_list, ignore_index=True)

In [ ]:
df

In [ ]:
pd.read_csv("/workspace/results/simulation/Brisbane_min_soc=0_max_soc=0.6_cooling_disabled=false/summary.csv")

In [ ]:
df = pd.read_csv("/workspace/results/simulation/Brisbane_min_soc=0_max_soc=0.6_cooling_disabled=false/results.csv")

In [ ]:
dt = 5/60

In [ ]:
df_disp

In [ ]:
batt.outputs['q_EFC'][-10:]

In [ ]:
battery.stressors["soc"][:10]

In [ ]:
battery.stressors["dod"][:10]

In [ ]:
from stages.pixii_blast import simulate_battery

In [ ]:
data = utils.generate_sample_data()

In [ ]:
data

In [ ]:

models.available_models()

In [ ]:
    params = pybamm.ParameterValues("Marquis2019")
    params.update({
        "Nominal cell capacity [A.h]": BLOCK_AH,  # from datasheet
        "Number of cells connected in series to make a battery": N_SERIES,     # per module
        "Upper voltage cut-off [V]": 3.65 * N_SERIES,  # 58.4 V per block
        "Lower voltage cut-off [V]": 2.8 * 16,   # 44.8 V per block
        "Ambient temperature [K]": ambient_temp_func,
    })

In [ ]:
tmy = pd.read_csv("data/tmy/Brisbane.csv", index_col="Datetime")
tmy.index= pd.to_datetime(tmy.index)

In [ ]:
tmy.index

In [ ]:
tmy

In [ ]:
start_time = '2025/09/22 20:00:00'
end_time = '2025/09/22 21:00:00'
table = 'DISPATCHPRICE'
raw_data_cache = 'cache'

price_data = dynamic_data_compiler(start_time, end_time, table, raw_data_cache, filter_cols=["REGIONID"], filter_values=[["QLD1"]])


In [ ]:
tmy_df.index.seconds

In [ ]:
pybamm.Interpolant(tmy.index.seconds, temp_K, pybamm.t), merged_temp

In [ ]:
price_data[['SETTLEMENTDATE', "REGIONID", "RRP"]]

In [ ]:
model.default_parameter_values.keys()

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def build_temperature_function(tmy_df: str, price_df: pd.DataFrame):
    """
    Build an interpolated ambient temperature function aligned with the dispatch data.

    Args:
        tmy_df: DataFrame containing TMY data for a arbitrary year.
        price_df: DataFrame containing the dispatch prices with datetime index (5 min intervals).

    Returns:
        temp_func: callable(t) returning temperature [K]
        temp_series: pandas Series of interpolated temperature [°C] aligned with price_df index
    """

    tmy_df = tmy_df.copy()
    # Normalize to a synthetic "year" (day-of-year & hour pattern)
    tmy_df["DOY"] = tmy_df.index.dayofyear
    tmy_df["TOD"] = tmy_df.index.hour + tmy_df.index.minute / 60.0

    # --- 2. Build the corresponding DOY/TOD for the dispatch timestamps
    price_df = price_df.copy()
    price_df["DOY"] = price_df.index.dayofyear
    price_df["TOD"] = price_df.index.hour + price_df.index.minute / 60.0

    # Merge based on DOY/TOD pattern
    # tmy_daily = tmy_df.set_index(["DOY", "TOD"])
    # merged_temp = (
    #     price_df.set_index(["DOY", "TOD"])
    #     .join(tmy_daily["temp_air"], on=["DOY", "TOD"], how="left")
    #     .interpolate(m)

    # --- Merge pattern data ---
    tmy_daily = tmy_df.set_index(["DOY", "TOD"])
    price_pattern = price_df.copy()
    price_pattern["DOY"] = price_pattern.index.dayofyear
    price_pattern["TOD"] = price_pattern.index.hour + price_pattern.index.minute / 60.0

    merged_temp = (
        price_pattern.set_index(["DOY", "TOD"])
        .join(tmy_daily["temp_air"], on=["DOY", "TOD"], how="left")
        .interpolate(method="linear", limit_direction="both")
    )

    # --- Force reindex to original datetime index ---
    merged_temp = merged_temp.reindex(price_df.index, method="nearest")
    # --- 3. Convert to Kelvin and interpolate over time (seconds)
    t_seconds = (merged_temp.index.view(np.int64) / 1e9).to_numpy()
    T_kelvin = merged_temp["temp_air"].to_numpy() + 273.15

    temp_interp = interp1d(t_seconds, T_kelvin, kind="linear", fill_value="extrapolate")

    def ambient_temp(t):
        """Return temperature in Kelvin at time t [seconds]."""
        return float(temp_interp(t))

    return ambient_temp, merged_temp["temperature_C"]

In [ ]:
def build_temperature_function(tmy_df, price_df):
    """
    Align hourly TMY temperature (and optionally humidity) with 5-min price data
    and return smooth interpolation functions in Kelvin.
    """

    # --- 1. Prepare and clean TMY ---
    tmy_df = tmy_df.copy()
    tmy_df.index = pd.to_datetime(tmy_df.index)
    tmy_df["DOY"] = tmy_df.index.dayofyear
    tmy_df["TOD"] = tmy_df.index.hour + tmy_df.index.minute / 60.0
    tmy_df.rename(columns={"temp_air": "temperature_C"}, inplace=True)

    # --- 2. Build daily/hour pattern from TMY ---
    tmy_daily = tmy_df.set_index(["DOY", "TOD"])

    # --- 3. Extract day/time-of-day pattern for price timestamps ---
    price_pattern = price_df.copy()
    price_pattern["DOY"] = price_pattern.index.dayofyear
    price_pattern["TOD"] = price_pattern.index.hour + price_pattern.index.minute / 60.0

    # --- 4. Join TMY data ---
    merged_temp = (
        price_pattern.set_index(["DOY", "TOD"])
        .join(tmy_daily[["temperature_C"]], on=["DOY", "TOD"], how="left")
        .interpolate(method="linear", limit_direction="both")
        .reset_index(drop=True)
    )

    # --- 5. Align with price_df index ---
    merged_temp.index = price_df.index  # now safe; same length & order

    # --- 6. Convert to Kelvin & build interpolators ---
    t_seconds = merged_temp.index.view(np.int64) / 1e9
    temp_K = merged_temp["temperature_C"].to_numpy() + 273.15

    # --- 7. Interpolating functions (Kelvin & relative humidity %) ---
    def ambient_temp(t):
        """Return ambient temperature (K) given timestamp in seconds"""
        return np.interp(t, t_seconds, temp_K, left=temp_K[0], right=temp_K[-1])

    print(f"✓ Aligned TMY with {len(price_df)} dispatch intervals.")
    print(f"Temperature range: {merged_temp['temperature_C'].min():.1f}–{merged_temp['temperature_C'].max():.1f} °C")

    return ambient_temp, merged_temp

In [ ]:
ambient_temp(2)

In [ ]:
aest_tz = pytz.timezone("Australia/Brisbane")

tmy_df = pd.read_csv("data/tmy/Brisbane.csv", parse_dates=["time(UTC)"])
tmy_df = tmy_df.rename(columns={"time(UTC)": "Datetime"})

tmy_df = tmy_df.set_index("Datetime")

# 2. Convert to AEST
tmy_df = tmy_df.tz_convert(aest_tz)

# 3. Price data should already be in AEST, so no conversion needed
price_df = pd.read_csv("data/dispatch/dispatch.csv", parse_dates=["SETTLEMENTDATE"])
price_df = price_df.set_index("SETTLEMENTDATE").sort_index()
price_df = price_df.tz_localize(aest_tz)

In [ ]:
ambient_temp, merged_temp = build_temperature_function(tmy_df, price_df)

In [ ]:
merged_temp

In [ ]:
price_data['SETTLEMENTDATE'] = pd.to_datetime(price_data["SETTLEMENTDATE"])
price_data = price_data.set_index("SETTLEMENTDATE")

In [ ]:
price_data.to_csv("dispatch_price.csv", index=True)

In [ ]:
price_data["RRP"].iloc[0]

In [ ]:
battery_capacity = 202600
P_max = 60000
efficiency = 0.969
max_doc = 0.1
max_hoc = 1
dt = 5/60

In [ ]:
model = g

In [ ]:
models = [
    pybamm.lithium_ion.SPM(),
    pybamm.lithium_ion.SPMe(),
    pybamm.lithium_ion.DFN(),
]

In [ ]:
sims = []
for model in models:
    sim = pybamm.Simulation(model)
    sim.solve([0, 3600])
    sims.append(sim)

In [ ]:
import pvlib, pytz

In [ ]:
lc = pvlib.location.Location(latitude=-27.4705, longitude=153.0260)
aest_tz = pytz.timezone("Australia/Brisbane")

tmy_df, _ = pvlib.iotools.get_pvgis_tmy(-27.4705, 153.0260)
tmy_df.index = tmy_df.index.tz_convert(aest_tz)

In [ ]:
tmy_df.groupby(tmy_df.index.hour).mean()['relative_humidity'].plot.line()